# Contour Extraction & Smoothing

Here we have an example of contour extraction from a folder of images. <br>

The inputs:
1. path name; *pth*: this is the location of the folder that contains the images which you want to extract contours from.
1. path for plots; _pth_for_plots_: the folder you want to save the new plots in (i.e. images with smoothed contours overlayed).
1. path for CSVs; _pth_for_CSVs_: the folder you want to save the CSVs of the smoothed contours in.

Note, the code assumes that you wish to name the columns in the CSV containing a contour as "x", "y", and "pointOrder". If you don't want this, then change these words in line 41 of the code in the __Contour Extraction Example__ tab below.

#### Imports

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from cont_extraction import binarize
from cont_extraction import get_outline_contour
from cont_extraction import load_resize_image
from cont_extraction import snakeSmooth
from cont_extraction import choose_contour

### Contour Extraction Example

In [6]:
filenames = []
smoothedContours = []
pth = "C:\\Users\\arian\\Downloads\\original images\\original images"
pth_for_plots = "C:\\Users\\arian\\Documents\\testPlots\\"
pth_for_CSVs = "C:\\Users\\arian\\Documents\\testCSVs\\"

for file in os.listdir(pth):
    if ("jpg" in file) or ("jpeg" in file) or ("png" in file):
        filenames.append(file)
        # 1) Load and resize image:
        img = load_resize_image(pth+"\\"+file,resize=0.8)
        # 2) Binarize
        mpx,mpy,finalImages,proportions = binarize(img,white_or_black='both')
        print("Getting contour for image "+file)
        # 3) Find outline contours:
        x1,y1 = get_outline_contour(finalImages[0],mpx,mpy)
        x2,y2 = get_outline_contour(finalImages[1],mpx,mpy)
        if (x1 == [0]) or (proportions[0]>0.98):
            print("Redoing contour for image "+file)
            x1,y1 = get_outline_contour(img,mpx,mpy)
        if (x2 == [0]) or (proportions[1]>0.98):
            print("Redoing contour for image "+file)
            x2,y2 = get_outline_contour(img,mpx,mpy)
        # 4) Choose best contour:
        x,y = choose_contour(x1,y1,x2,y2)
        # 5) Basic contour smoothing:
        xs,ys = snakeSmooth(img,x,y)
        # 6) Save contours into list:
        smoothedContours.append([xs,ys])
        # 7) Plot contours overlayed on images:
        fig,ax = plt.subplots()
        ax.imshow(img,cmap='gray')
        ax.plot(xs,ys,'-r',linewidth=2)
        ax.axis('off')
        plt.savefig(pth_for_plots+'Contour_'+file,bbox_inches = 'tight',pad_inches=0.0)
        plt.close()
        # 8) Save CSV of contour:
        contourCSV = pd.DataFrame([xs,ys,list(range(1,len(ys)+1))]).T
        contourCSV  = contourCSV.rename(columns={0:'x',1:'y',2:'pointOrder'})
        contourCSV.to_csv(pth_for_CSVs+file[:-3]+'_smoothedContour.csv',index=False)        
    
print("Extracted contours from "+str(n)+" images!")

Getting contour for image pot_number1002098.0_url.img_5.jpg
Getting contour for image pot_number1006198.0_url.img_8.jpg
Getting contour for image pot_number1009549.0_url.img_5 copy.jpg
Getting contour for image pot_number180001.0_url.img_6 copy.jpg
Getting contour for image pot_number23074.0_url.img_7.jpg
Extracted contours from 5 images!
